# Word Embedded Bidirectional LSTM Siamese Network Export

This notebook shows how to export a trained sentence similarity/sentence embedding model using word embedding and bidirectional LSTM on a siamese network. In general though it is easier to export after the training but in some cases people might want to separate the process

## Import the needed modules

In [ ]:
import os
import tensorflow as tf
import kotoba as kt
import narau as nr
import cloudpickle

## Define data related constants

*   ```MODEL_DIR```: Directory of the model to export
*   ```EMBEDDING_FILE```: Path to the Glove embedding file

In [ ]:
MODEL_DIR = 'word_bilstm_glove_siamese\\model\\1536902903'
EMBEDDING_FILE = os.path.expanduser('~/Documents/data/glove.6B/glove.6B.100d.txt')

## Define model related constants

*   ```EMBEDDING_WEIGHTS```: Initial weights of the embedding layer
*   ```EMBEDDING_SIZE```: Number of tokens in the embedding
*   ```EMBEDDING_DIMENSION```: Number of dimensions in the dense representation
*   ```EMBEDDING_SPECIAL_TOKENS```: Number of special tokens
*   ```EMBEDDING_WITH_PAD```: If embedding includes a padding
*   ```EMBEDDING_TRAINABLE```: If embedding is trainable
*   ```EMBEDDING_UNITS```: List of units in the embedding transform function
*   ```LSTM_UNITS```: List of the LSTM units
*   ```LSTM_DROPOUT```: LSTM output dropout probability
*   ```PROJECTION_UNITS```: List of units of the projection. Last item determines embedding dimensions
*   ```LOSS_MARGIN```: Target distance between different sentences
*   ```LEARNING_RATE```: Rate of gradient application

Note: There is no guarantee if it would work if the parameters are different so they are kept the same. However it is recommended to just inherit the model and override its constructor arguments so that there is no need to redefine these constant in two separate scripts/notebooks

In [ ]:
EMBEDDING_WEIGHTS = nr.embedding.load_glove_weights(EMBEDDING_FILE)
EMBEDDING_SIZE = EMBEDDING_WEIGHTS.shape[0]
EMBEDDING_DIMENSION = EMBEDDING_WEIGHTS.shape[1]
EMBEDDING_SPECIAL_TOKENS = 2
EMBEDDING_WITH_PAD = True
EMBEDDING_TRAINABLE = False

EMBEDDING_UNITS = [128]

LSTM_UNITS = [128]
LSTM_DROPOUT = 0.5

PROJECTION_UNITS = [1024]

LOSS_MARGIN = 1.0
LEARNING_RATE = 0.1

## Create a preprocessing pipeline
A pipeline is created for the preprocessing of the model. This is because some of the preprocessing is done outside tensorflow. A pipeline that directly converts the text to a serialized TFRecord is created using kotoba. The logic is similar to the preprocessing done during the training. This time however, there is no label to process.

In [ ]:
class Preprocessor(kt.Preprocessor):
    
    def __init__(self):
        self._pipeline = kt.Pipeline([
            kt.LowerCase(),
            kt.tokenizer.NLTKTokenizer(),
            kt.embedding.EmbedTokenToID(
                kt.embedding.Embedding.from_glove_file(EMBEDDING_FILE, ['<PAD>', '<UNK>'], 1)
            ),
        ])
        
    def transform(self, x, as_iterable=False):
        x_pipelined = self._pipeline.transform(x, True)
        example = nr.example.SequenceExample(
            nr.example.FeatureLists({
                'x': nr.example.Int64FeatureList(x_pipelined)
            })
        )
        return example.SerializeToString()

## Creating the serving API
To serve a model, the signature for using the model must be defined. Here a function that returns a ```ServingInputReceiver``` must be defined. In that function, the processing to load the data to the model must also be defined. The loading processing is similar to the training process. The only difference is that there are no labels this time.

References:
*   ```Exporting Models```: https://www.tensorflow.org/guide/saved_model#using_savedmodel_with_estimators
*   ```ServingInputReceiver```: https://www.tensorflow.org/api_docs/python/tf/estimator/export/ServingInputReceiver

In [ ]:
_key = 'x'

_feature_def = {
    _key: tf.VarLenFeature(tf.int64),
}

def parse_example(example):
    context, features = tf.parse_single_sequence_example(example, sequence_features=_feature_def)
    return features

def preprocess_text(x):
    x = tf.sparse_reset_shape(x)
    x = tf.sparse_tensor_to_dense(x)
    return x, tf.size(x)

def preprocess_elements(features):
    x, l = preprocess_text(features[_key])
    return (x, l)

def input_fn(serialized_example):
    parsed_example = parse_example(serialized_example)
    ds = tf.data.Dataset.from_tensor_slices(parsed_example)
    ds = ds.map(preprocess_elements, num_parallel_calls=8)
    ds = ds.padded_batch(parsed_example[_key].dense_shape[0], ([None], []))
    it = ds.make_initializable_iterator()
    with tf.control_dependencies([it.initializer]):
        data = it.get_next()
    return data

def serving_input_receiver_fn():
    serialized_example = tf.placeholder(tf.string, [])
    receiver_tensors = {'input': serialized_example}
    features = input_fn(serialized_example)
    features_dict = {
        'x': features[0],
        'len': features[1],
    }
    return tf.estimator.export.ServingInputReceiver(features_dict, receiver_tensors)

## Defining the session and run configurations
The model should not need these settings since the model will not be executed. However the same config with the training is used here just in case.

In [ ]:
session_config = tf.ConfigProto()
session_config.allow_soft_placement = True
session_config.gpu_options.allow_growth = True

config = tf.estimator.RunConfig(tf_random_seed=None,
                                save_summary_steps=50,
                                save_checkpoints_steps=400,
                                keep_checkpoint_max=None,
                                session_config=session_config)

## Defining the model
It is recommended that the model is subclassed so that the constants doesn't have to be defined everytime. However for this code the settings in the training is directly copied to the export to achieve the same result.

In [ ]:
clf = nr.estimators.SiameseBiLSTMEmbedding(EMBEDDING_SIZE, 
                                           EMBEDDING_DIMENSION, 
                                           EMBEDDING_SPECIAL_TOKENS,
                                           EMBEDDING_WITH_PAD, 
                                           EMBEDDING_WEIGHTS,
                                           EMBEDDING_TRAINABLE, 
                                           EMBEDDING_UNITS,
                                           LSTM_UNITS,
                                           LSTM_DROPOUT,
                                           PROJECTION_UNITS,
                                           LOSS_MARGIN,
                                           LEARNING_RATE,
                                           MODEL_DIR,
                                           config)

## Exporting the model
Export the model by providing an output path and the function that return the ```ServingInputReceiver```. This will create a file that can be used to deploy the model

In [ ]:
export_path = clf.export_savedmodel('word_bilstm_glove_siamese\\savedmodel\\1536902903', serving_input_receiver_fn)
export_path

## Exporting the Preprocessor
The preprocessor is exported after the model so that the name of the path for the preprocessor is similar to the path of the exported model

In [ ]:
prep_path = export_path + b'_prep.pkl'
with open(prep_path, 'wb') as file:
    cloudpickle.dump(prep, file)